<a href="https://colab.research.google.com/github/RocioLiu/Python_coding/blob/master/BERT_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Refer to  [進擊的 BERT：NLP 界的巨人之力與遷移學習](https://leemeng.tw/attack_on_bert_transfer_learning_in_nlp.html)

In [1]:
%%bash
pip install transformers tqdm boto3 requests regex -q

ERROR: botocore 1.20.84 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.


In [2]:
import torch
from transformers import BertTokenizer
from IPython.display import clear_output

In [3]:
PRETRAINED_MODEL_NAME = "bert-base-chinese" # 指定繁簡中文 BERT-BASE 預訓練模型

# 取得此預訓練模型所使用的 tokenizer
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

In [4]:
clear_output()
print("PyTorch 版本：", torch.__version__)

PyTorch 版本： 1.8.1+cu101


In [5]:
# 看看 tokenizer 裡頭的字典資訊
vocab = tokenizer.vocab
print(f"字典大小：{len(vocab)}")

字典大小：21128


如上所示，中文 BERT 的字典大小約有 2.1 萬個 tokens

In [6]:
import random
random_tokens = random.sample(list(vocab), 10)
random_tokens

['邓', '##赣', '245', '##咙', '鏈', 'amazon', '##啡', '攣', 'live', '嘶']

In [7]:
random_ids = [vocab[t] for t in random_tokens]
random_ids

[6924, 19675, 10325, 14536, 7122, 8954, 14622, 3112, 8582, 1674]

In [8]:
print(f"{'token': <20}{'index':<15}")
print('-'*25)
for t, id in zip(random_tokens, random_ids):
  print(f"{t:<20}{id:<10}")

token               index          
-------------------------
邓                   6924      
##赣                 19675     
245                 10325     
##咙                 14536     
鏈                   7122      
amazon              8954      
##啡                 14622     
攣                   3112      
live                8582      
嘶                   1674      


BERT 使用當初 [Google NMT](https://ai.googleblog.com/2016/09/a-neural-network-for-machine.html) 提出的 [WordPiece Tokenization](https://arxiv.org/abs/1609.08144) ，將本來的 words 拆成更小粒度的 wordpieces，有效處理不在字典裡頭的詞彙 。中文的話大致上就像是 character-level tokenization，而有 ## 前綴的 tokens 即為 wordpieces。

ㄅㄆㄇㄈ也有被收錄：

In [9]:
indices = list(range(647, 657))
some_pairs = [(t, idx) for t, idx in vocab.items() if idx in indices]
for pair in some_pairs:
  print(pair)

('ㄅ', 647)
('ㄆ', 648)
('ㄇ', 649)
('ㄉ', 650)
('ㄋ', 651)
('ㄌ', 652)
('ㄍ', 653)
('ㄎ', 654)
('ㄏ', 655)
('ㄒ', 656)


利用中文 BERT 的 tokenizer 將一個中文句子斷詞看看：

In [10]:
text = "[CLS] 等到潮水 [MASK] 了，就知道誰沒穿褲子。"
tokens = tokenizer.tokenize(text)
ids = tokenizer.convert_tokens_to_ids(tokens)

print(text)
print(tokens)
print(ids)

[CLS] 等到潮水 [MASK] 了，就知道誰沒穿褲子。
['[CLS]', '等', '到', '潮', '水', '[MASK]', '了', '，', '就', '知', '道', '誰', '沒', '穿', '褲', '子', '。']
[101, 5023, 1168, 4060, 3717, 103, 749, 8024, 2218, 4761, 6887, 6306, 3760, 4959, 6194, 2094, 511]


除了一般的 wordpieces 以外，BERT 裡頭有 5 個特殊 tokens：
* [CLS]：在做分類任務時其最後一層的 repr. 會被視為整個輸入序列的 repr.
* [SEP]：有兩個句子的文本會被串接成一個輸入序列，並在兩句之間插入這個 token 以做區隔
* [UNK]：沒出現在 BERT 字典裡頭的字會被這個 token 取代
* [PAD]：zero padding 遮罩，將長度不一的輸入序列補齊方便做 batch 運算
* [MASK]：未知遮罩，僅在預訓練階段會用到

如上例所示，[CLS] 一般會被放在輸入序列的最前面，而 zero padding 在之前的 [Transformer 文章](https://leemeng.tw/neural-machine-translation-with-transformer-and-tensorflow2.html#%E7%9B%B4%E8%A7%80%E7%90%86%E8%A7%A3%E9%81%AE%E7%BD%A9%E5%9C%A8%E6%B3%A8%E6%84%8F%E5%87%BD%E5%BC%8F%E4%B8%AD%E7%9A%84%E6%95%88%E6%9E%9C) 裡已經有非常詳細的介紹。[MASK] token 一般在 fine-tuning 或是 feature extraction 時不會用到，這邊只是為了展示預訓練階段的克漏字任務才使用的。

現在讓我們看看給定上面有 [MASK] 的句子，BERT 會填入什麼字：

In [11]:
"""
這段程式碼載入已經訓練好的 masked 語言模型並對有 [MASK] 的句子做預測
"""
from transformers import BertForMaskedLM
# 除了 tokens 以外我們還需要辨別句子的 segment ids
tokens_tensor = torch.tensor([ids]) # (1, seq_len)
segments_tensors = torch.zeros_like(tokens_tensor) # (1, seq_len)
maskedLM_model = BertForMaskedLM.from_pretrained(PRETRAINED_MODEL_NAME)
clear_output()

In [12]:
tokens_tensor

tensor([[ 101, 5023, 1168, 4060, 3717,  103,  749, 8024, 2218, 4761, 6887, 6306,
         3760, 4959, 6194, 2094,  511]])

In [15]:
# 使用 masked LM 估計 [MASK] 位置所代表的實際 token
maskedLM_model.eval()
with torch.no_grad():
  outputs = maskedLM_model(tokens_tensor, segments_tensors)
  predictions = outputs[0] # (1, seq_len, num_hidden_units)

In [20]:
outputs

MaskedLMOutput([('logits',
                 tensor([[[ -9.9042,  -9.5542,  -9.8265,  ...,  -6.8384,  -6.1913,  -6.8853],
                          [ -8.5631,  -7.9794,  -8.6885,  ...,  -5.8659,  -4.6942,  -5.3190],
                          [-16.6086, -16.1292, -16.4605,  ..., -12.4272, -10.7403, -15.4416],
                          ...,
                          [-12.6074, -13.6432, -13.6270,  ..., -11.8602,  -8.6505, -15.0264],
                          [-14.8105, -15.8511, -15.1675,  ..., -10.1823,  -1.8096, -11.0179],
                          [-12.4678, -12.5876, -12.1389,  ...,  -9.1602,  -3.3892,  -9.0933]]]))])

In [36]:
predictions

tensor([[[ -9.9042,  -9.5542,  -9.8265,  ...,  -6.8384,  -6.1913,  -6.8853],
         [ -8.5631,  -7.9794,  -8.6885,  ...,  -5.8659,  -4.6942,  -5.3190],
         [-16.6086, -16.1292, -16.4605,  ..., -12.4272, -10.7403, -15.4416],
         ...,
         [-12.6074, -13.6432, -13.6270,  ..., -11.8602,  -8.6505, -15.0264],
         [-14.8105, -15.8511, -15.1675,  ..., -10.1823,  -1.8096, -11.0179],
         [-12.4678, -12.5876, -12.1389,  ...,  -9.1602,  -3.3892,  -9.0933]]])

In [24]:
predictions.size()

torch.Size([1, 17, 21128])

In [21]:
del maskedLM_model

In [35]:
# 將 [MASK] 位置的機率分佈取 top k 最有可能的 tokens 出來
masked_index = 5  # [mask] 位於 index 5
k = 3
probs, indices = torch.topk(torch.softmax(predictions[0, masked_index], -1), k)
predicted_tokens = tokenizer.convert_ids_to_tokens(indices.tolist())